In [3]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry

In [8]:
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [9]:
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 50.4547,
	"longitude": 30.5238,
	"hourly": ["temperature_2m", "wind_speed_10m"],
	"timezone": "Europe/Berlin"
}
responses = openmeteo.weather_api(url, params=params)

In [10]:
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates 50.4375°N 30.5°E
Elevation 188.0 m asl
Timezone b'Europe/Berlin'b'GMT+2'
Timezone difference to GMT+0 7200 s


In [11]:
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(1).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

                         date  temperature_2m  wind_speed_10m
0   2025-06-16 22:00:00+00:00       19.359501        3.415260
1   2025-06-16 23:00:00+00:00       18.759501        3.671294
2   2025-06-17 00:00:00+00:00       17.859501        3.600000
3   2025-06-17 01:00:00+00:00       17.109501        3.976330
4   2025-06-17 02:00:00+00:00       16.659500        3.758510
..                        ...             ...             ...
163 2025-06-23 17:00:00+00:00       24.521000        0.360000
164 2025-06-23 18:00:00+00:00       23.320999        2.811690
165 2025-06-23 19:00:00+00:00       21.771000        3.826853
166 2025-06-23 20:00:00+00:00       20.021000        4.510787
167 2025-06-23 21:00:00+00:00       18.620998        4.452954

[168 rows x 3 columns]


In [ ]:
df_3days=hourly_dataframe.iloc[0:72]
min_temperature_3days=df_3days["temperature_2m"].min()
max_temperature_3days=df_3days["temperature_2m"].max()
mean_temperature_3days=df_3days["temperature_2m"].mean()
wind_spead_greater_mean=hourly_dataframe["wind_speed_10m"] > hourly_dataframe["wind_speed_10m"].mean()
hours_wind_spead_greater_mean=wind_spead_greater_mean.value_counts().get(True)




74